In [ ]:
# =========================
# 0) 설치 (Colab 1회)
# =========================
!pip -q install deep_sort_realtime opencv-python
!git clone -q https://github.com/ultralytics/yolov5
%cd yolov5
!pip -q install -r requirements.txt

import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

# =========================
# 1) YOLOv5s 로드
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"
yolo = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)
yolo.to(device)
yolo.conf = 0.4      # confidence threshold
yolo.iou  = 0.45

# =========================
# 2) DeepSORT 로드 (가볍게)
# =========================
# embedder는 가벼운 mobilenet 계열이 기본으로 잘 동작함
tracker = DeepSort(
    max_age=20,
    n_init=2,                # ✅ 동영상이면 2~3이 보통 안정적 (1은 첫 프레임 confirmed=0이 자주 나옴)
    nms_max_overlap=1.0
)

# =========================
# 3) 테스트 동영상 준비 (여기만 네 mp4로 바꾸면 됨)
# =========================
video_path = "/content/test.mp4"   # ✅ 네가 다운받아 둔 동영상 경로로 바꿔
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "동영상 로드 실패. video_path 확인해봐."

fps = cap.get(cv2.CAP_PROP_FPS)
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out_path = "/content/out_track.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(out_path, fourcc, fps if fps > 0 else 30.0, (w, h))

frame_idx = 0
while True:
    ret, frame_bgr = cap.read()
    if not ret:
        break
    frame_idx += 1

    # =========================
    # 4) YOLO 추론 (프레임 단위)
    # =========================
    results = yolo(frame_bgr, size=640)
    det = results.xyxy[0].detach().cpu().numpy()   # [x1,y1,x2,y2,conf,cls]

    # 사람만 쓰고 싶으면 클래스 0(person)만 필터
    # COCO에서 person=0
    dets_for_deepsort = []
    yolo_boxes = []  # ✅ confirmed가 없을 때도 YOLO 박스 시각화용
    for x1, y1, x2, y2, conf, cls in det:
        cls = int(cls)
        if cls != 0:   # person만
            continue
        x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
        conf = float(conf)

        yolo_boxes.append((int(x1), int(y1), int(x2), int(y2), conf))

        # deep_sort_realtime 입력 포맷:
        # [ [x1,y1,w,h], confidence, class_name ]
        dets_for_deepsort.append(([x1, y1, x2-x1, y2-y1], conf, "person"))

    # =========================
    # 5) DeepSORT 업데이트 (프레임 단위)
    # =========================
    tracks = tracker.update_tracks(dets_for_deepsort, frame=frame_bgr)

    out = []
    for t in tracks:
        if not t.is_confirmed():
            continue
        track_id = t.track_id
        l, ttop, r, b = t.to_ltrb()
        out.append((track_id, (int(l), int(ttop), int(r), int(b))))

    # =========================
    # 6) 시각화 후 저장 (동영상)
    # =========================
    vis = frame_bgr.copy()

    # ✅ 1) DeepSORT confirmed track이 있으면 초록 박스(ID)
    for track_id, (l, ttop, r, b) in out:
        cv2.rectangle(vis, (l, ttop), (r, b), (0,255,0), 2)
        cv2.putText(vis, f"ID {track_id}", (l, max(0, ttop-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    # ✅ 2) confirmed가 아직 없을 때도 YOLO 결과(빨간 박스) 표시해서 “아무것도 안 보임” 방지
    if len(out) == 0:
        for (x1, y1, x2, y2, conf) in yolo_boxes:
            cv2.rectangle(vis, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.putText(vis, f"YOLO {conf:.2f}", (x1, max(0, y1-7)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    cv2.putText(vis, f"frame {frame_idx} | yolo={len(dets_for_deepsort)} | confirmed={len(out)}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    writer.write(vis)

cap.release()
writer.release()
print("Saved:", out_path)

# Colab에서 파일 다운로드 링크
from google.colab import files
files.download(out_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C
^C
[Errno 2] No such file or directory: 'yolov5'
/content
^C
